# 'Querdenker' vor und nach Covid-19 - eine Untersuchung mit Word2Vec

#### Bedeutung des Lemmas 'Querdenker'

Person, die eigenwillige und mit etablierten Positionen meist nicht vereinbare Ideen oder Ansichten vertritt, äußert und deshalb oft auf Unverständnis oder Widerstand trifft <br>
(Quelle: https://www.dwds.de/wb/Querdenker) <br>
<br>
[spezieller] in der COVID-19-Pandemie: Person, die die Coronamaßnahmen für überzogen hält und sich dabei zum Teil auf wissenschaftliche Minderheitenmeinungen beruft, vor allem aber solche Informationen zur Bekräftigung ihrer Überzeugung heranzieht, die bei Experten als unsachlich bzw. falsch oder als zu stark vereinfacht gelten <br>
(Quelle: https://www.dwds.de/themenglossar/Corona#glossar-Q) <br>
 

#### Hypothese

Durch die Pandemie und die daraus entstandenen Bewegung der sogenannten Querdenker wird das Lemma populärer, die Bedeutung spezifiziert sich und verdrängt die ursprüngliche Bedeutung. Die Bedeutung des Lemmas wandelt sich und wird möglicherweise negativ konnotiert. 

## Importe und Datenvorbereitung

In [1]:
import codecs
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import seaborn as sns
import spacy
import statistics 

from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases
from joblib import Parallel, delayed  
from nltk.corpus import stopwords
from scipy import spatial
from sklearn.manifold import TSNE
from tabulate import tabulate

In [2]:
nlp = spacy.load('de_core_news_md')
stopwords = stopwords.words('german')
tokenizer = nltk.data.load('tokenizers/punkt/german.pickle')

#### Korpus laden und teilen

In [3]:
# die Datei enthält alle Treffer des Lemmas 'querdenker' im ZDL-Regionalkorpus des DWDS 
# mit jeweils einem Satz Kontext davor und danach

df = pd.read_csv('../data/querdenker_1993-2021.csv', sep=',', encoding='utf-8')
df.head()

,Corpus,Date,Genre,Bibl,ContextBefore,Hit,ContextAfter
0,saar_regional,1993-04-28,Zeitung,"Saarbrücker Zeitung, 28.04.1993","[...]andes war, bedankte sich Sauer mit einer ...","Gerade Thiery als ""Querdenker"" habe mit seinen...","Edgar Schuster, Ortsvorsteher von Noswendel un..."
1,saar_regional,1993-07-09,Zeitung,"Saarbrücker Zeitung, 09.07.1993","[...]tellvertretend für die zwei ""Lager"" äußer...","Der Ehrenvorsitzende der Völklinger CDU, Dr. R...",Angesichts der schon in einem Jahr stattfinde[...
2,saar_regional,1993-09-02,Zeitung,"Saarbrücker Zeitung, 02.09.1993","[...]ant zu Heinrich Böll ist, oder ob ihn die...","""Ich fühle mich geschmeichelt"", so der gerührt...","Dennoch, konstatierte Ratlosigkeit auch bei Wo..."
3,saar_regional,1993-10-15,Zeitung,"Saarbrücker Zeitung, 15.10.1993",Ab sofort erhältlich,"Schade, daß das Buch auch auf die Rolle von Fr...",Die Neuerscheinung jed[...]
4,saar_regional,1993-11-12,Zeitung,"Saarbrücker Zeitung, 12.11.1993","[...] profilierten Politikern, die auch einmal...",Querdenker erzielen aber nie hundertprozentige...,Und auch bei der anstehenden Besetzung der Plä...


In [4]:
# NaN durch Whitespace ersetzen

df = df.fillna(' ')

In [ ]:
# Wörter, die mit [...] anfangen löschen

expression = '\[...]\w*'
df = df.replace(to_replace = expression, value = ' ', regex=True)

In [5]:
# bevor die Spalten verbunden werden: Whitespace einfügen, um ein Aneinanderkleben der Wörter zu verhindern

df['ContextBefore'] = df['ContextBefore'].astype(str) + ' '
df['ContextAfter'] = ' ' + df['ContextAfter'].astype(str) 

In [6]:
# Text zu einer Spalte verbinden

columns = ['ContextBefore', 'Hit', 'ContextAfter']

df['Text'] = df[columns].astype(str).sum(axis=1)
df.head()

,Corpus,Date,Genre,Bibl,ContextBefore,Hit,ContextAfter,Text
0,saar_regional,1993-04-28,Zeitung,"Saarbrücker Zeitung, 28.04.1993","[...]andes war, bedankte sich Sauer mit einer ...","Gerade Thiery als ""Querdenker"" habe mit seinen...","Edgar Schuster, Ortsvorsteher von Noswendel u...","[...]andes war, bedankte sich Sauer mit einer ..."
1,saar_regional,1993-07-09,Zeitung,"Saarbrücker Zeitung, 09.07.1993","[...]tellvertretend für die zwei ""Lager"" äußer...","Der Ehrenvorsitzende der Völklinger CDU, Dr. R...",Angesichts der schon in einem Jahr stattfinde...,"[...]tellvertretend für die zwei ""Lager"" äußer..."
2,saar_regional,1993-09-02,Zeitung,"Saarbrücker Zeitung, 02.09.1993","[...]ant zu Heinrich Böll ist, oder ob ihn die...","""Ich fühle mich geschmeichelt"", so der gerührt...","Dennoch, konstatierte Ratlosigkeit auch bei W...","[...]ant zu Heinrich Böll ist, oder ob ihn die..."
3,saar_regional,1993-10-15,Zeitung,"Saarbrücker Zeitung, 15.10.1993",Ab sofort erhältlich,"Schade, daß das Buch auch auf die Rolle von Fr...",Die Neuerscheinung jed[...],"Ab sofort erhältlich Schade, daß das Buch auch..."
4,saar_regional,1993-11-12,Zeitung,"Saarbrücker Zeitung, 12.11.1993","[...] profilierten Politikern, die auch einmal...",Querdenker erzielen aber nie hundertprozentige...,Und auch bei der anstehenden Besetzung der Pl...,"[...] profilierten Politikern, die auch einmal..."


In [7]:
df.shape

(7549, 8)

In [12]:
# Korpus in 2 Teilkorpora splitten: vor und nach Covid-19 (dem Ausbruch in Deutschland)
# hier wurde die Grenze zwischen Januar und Februar 2020 gezogen (es wird sich noch zeigen, inwiefern sich das bewährt)

df_before_corona = df.iloc[:4420,:]
df_after_corona = df.iloc[4421:,:]

In [13]:
df_after_corona.head()

,Corpus,Date,Genre,Bibl,ContextBefore,Hit,ContextAfter,Text
4421,laz_regional,2020-02-01,Zeitung,"Landshuter Zeitung, 01.02.2020",[...]wie einen Familienschatz:,Es sind beeindruckende Dokumente eines unbeugs...,Die Veröffentlichung der [...],[...]wie einen Familienschatz: Es sind beeindr...
4422,ta_regional,2020-02-01,Zeitung,"Thüringer Allgemeine, 01.02.2020",Essen und trinken in der Kirche,Vogelsberger Kirchgemeinde und Querdenker halt...,Annett Kletzke,Essen und trinken in der Kirche Vogelsberger K...
4423,mume_regional,2020-02-04,Zeitung,"Münchner Merkur, 04.02.2020",[...]n wie den Familienschatz:,Es sind beeindruckende Dokumente eines unbeugs...,Die Veröffentlichung der [...],[...]n wie den Familienschatz: Es sind beeindr...
4424,frt_regional,2020-02-06,Zeitung,"Fränkischer Tag, 06.02.2020",Danach werde für ihn als Bürgermeister definit...,Pöhnlein hat sich über die Jahre einen Namen a...,Immer wieder durchstreift er im Urlaub die Bu...,Danach werde für ihn als Bürgermeister definit...
4425,frt_regional,2020-02-06,Zeitung,"Fränkischer Tag, 06.02.2020","Diskussionsscheu werde er aber nicht sein, ver...",Denn vor dem Leben nach der Politik würde er f...,,"Diskussionsscheu werde er aber nicht sein, ver..."


#### Hilfsfunktionen zur Vorbereitung der Texte

In [14]:
def lemmatize_text_column(df, column):
    """
    transforms the Dataframe-column in a lemmatized string
    """
    text = ''
    for i in df[column]:
        doc = nlp(i)
        lemmas = ' '.join([x.lemma_ for x in doc])
        text = text + lemmas
    return text


def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_äÄöÖüÜß]',' ', raw).split()
    filtered_text = [word for word in text if word not in stopwords]
    return filtered_text


def prepare_text(raw_text):
    """
    returns a list of tokenized sentences
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    phrases = Phrases(tokenized_sentences)
    bigram = Phraser(phrases)
    sentences = list(bigram[tokenized_sentences])
    return sentences

#### Vorbereitung des ersten Texts (vor Covid-19)

In [15]:
text = lemmatize_text_column(df_before_corona, 'Text')
sentences = prepare_text(text)

# sentences ist eine Liste von tokenisierten Sätzen, zum Beispiel:
print(sentences[15])

['ideenreiche', 'innovative', 'firma', 'brache', 'heimat', 'finden', 'gelingen', 'branchen', 'mix', 'installieren', 'raum', 'querdenker', 'kreativ', 'lassen', 'bringen', 'gründerzentrum', 'stadt', 'völklingen', 'erhoffen', 'neu', 'impuls']


#### Vorbereitung des zweiten Texts (nach Covid-19)

In [16]:
text2 = lemmatize_text_column(df_after_corona, 'Text')
sentences2 = prepare_text(text2)

print(sentences2[100])

['zeitgleich', 'aktion', 'oma_rechts', 'finden', 'steinwurf', 'entfernen', 'münsterplatz', 'erneute', 'anti_corona', 'kundgebung', 'querdenker', 'schwarzwald', 'baar', 'statt']


## Training von Word2Vec auf den Text vor Covid-19

In [17]:
# Paramter setzen
workers = 4                      # Use these many worker threads to train the model (=faster training with multicore machines)
seed = 42                        # Seed for the random number generator

In [18]:
# Ordner anlegen zum Abspeichern von trainierten Modellen
if not os.path.exists('../trained_models'):
    os.makedirs('../trained_models')

In [19]:
# Training

w2v_bc = Word2Vec(sentences=sentences,                   
                 vector_size=300,          # Dimensionality of the word vectors
                 window=10,                # The maximum distance between the current and predicted word within a sentence
                 min_count=3,              # (int, optional) – The model ignores all words with total frequency lower than this
                 workers=workers, 
                 min_alpha=0.0001,         # Learning rate will linearly drop to min_alpha as training progresses
                 sg=1,                     # Training algorithm: skip-gram if sg=1, otherwise CBOW
                 seed=seed)

## Training von Word2Vec auf den Text nach Covid-19

In [20]:
w2v_ac = Word2Vec(sentences=sentences2,                   
                 vector_size=300,                
                 window=10,              
                 min_count=3,             
                 workers=workers, 
                 min_alpha=0.0001,                                                    
                 sg=1,                     
                 seed=seed)

In [21]:
# trainierte Modelle speichern
w2v_bc.save(os.path.join('../trained_models', 'w2v_bc_querdenker.model'))
w2v_ac.save(os.path.join('../trained_models', 'w2v_ac_querdenker.model'))

## Exploration und Vergleich der Embeddings

In [22]:
# trainierte Modelle laden
w2v_bc = Word2Vec.load(os.path.join('../trained_models', 'w2v_bc_querdenker.model'))
w2v_ac = Word2Vec.load(os.path.join('../trained_models', 'w2v_ac_querdenker.model'))

In [23]:
# ähnliche Wörter zu 'querdenker' vor Covid-19
w2v_bc.wv.most_similar(positive=['querdenker'], topn=25)

[('wählen', 0.9859026074409485),
 ('richtig', 0.9857795238494873),
 ('a', 0.9855211973190308),
 ('freiheit', 0.9852800965309143),
 ('sterben', 0.9848908185958862),
 ('moderator_politisch', 0.9847242832183838),
 ('lächerliche', 0.9846087098121643),
 ('reaktion', 0.9844648241996765),
 ('s', 0.9844154715538025),
 ('gleichschritt', 0.984307050704956),
 ('treiben', 0.9842811226844788),
 ('monieren', 0.9842720031738281),
 ('verlag', 0.9842686057090759),
 ('spaß', 0.9842213988304138),
 ('dichtung', 0.9841417670249939),
 ('goethe', 0.9841011166572571),
 ('leib', 0.9840785264968872),
 ('rainer', 0.9840254187583923),
 ('beispielsweise', 0.9839633107185364),
 ('campus', 0.9838644862174988),
 ('nabburg', 0.9838445782661438),
 ('ecke_kante', 0.9837514162063599),
 ('alt', 0.9837014079093933),
 ('rente', 0.9834622144699097),
 ('bedanken', 0.9834208488464355)]

In [24]:
# ähnliche Wörter zu 'querdenker' nach Covid-19
w2v_ac.wv.most_similar(positive=['querdenker'], topn=25)

[('märz', 0.9979901909828186),
 ('ernannt_querdenker', 0.997931182384491),
 ('querdenken', 0.9979155659675598),
 ('regensburg', 0.9978930950164795),
 ('angemeldet', 0.9978697299957275),
 ('dezember', 0.9978513717651367),
 ('ring', 0.9978469014167786),
 ('bayerische_verwaltungsgerichtshof', 0.9978463053703308),
 ('cockerwiese', 0.9978451728820801),
 ('initiative_querdenken', 0.9978318214416504),
 ('spätestens', 0.9978219866752625),
 ('gerichtlich', 0.9978201389312744),
 ('samstagnachmittag', 0.9978130459785461),
 ('augustusplatz', 0.997808575630188),
 ('verboten', 0.997793436050415),
 ('mehrer_tausend', 0.9977900385856628),
 ('behörde', 0.997784435749054),
 ('gericht', 0.997765302658081),
 ('oktober', 0.997764527797699),
 ('corona_schutzmaßnahmen', 0.9977623224258423),
 ('kommend', 0.9977535009384155),
 ('maßnahmen', 0.9977470636367798),
 ('anmeldung', 0.9977403283119202),
 ('schutzmaßnahmen', 0.9977369904518127),
 ('ordnungsamt', 0.9977298378944397)]

In [25]:
w2v_ac.wv.most_similar(positive=['virus'], topn=10)

[('denken', 0.9989206194877625),
 ('idee', 0.9988391399383545),
 ('glauben', 0.9988346099853516),
 ('kampf', 0.9987825751304626),
 ('natürlich', 0.9987759590148926),
 ('freiheit', 0.9987748861312866),
 ('fehlen', 0.9987468719482422),
 ('verbreiten', 0.9987419843673706),
 ('pandemie', 0.9987401366233826),
 ('neue', 0.9987374544143677)]

## Ausrichtung der beiden Embedding-Modelle

In [26]:
# Quelle: https://gist.github.com/zhicongchen/9e23d5c3f1e5b1293b16133485cd17d8

def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
    """
    Original script: https://gist.github.com/quadrismegistus/09a93e219a6ffc4f216fb85235535faf
    Updatet script: https://gist.github.com/zhicongchen/9e23d5c3f1e5b1293b16133485cd17d8
    Procrustes align two gensim models (to allow for comparison between same word across models).
    Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
        
    First, intersect the vocabularies (see `intersection_align_gensim` documentation).
    Then do the alignment on the other_embed model.
    Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
    Return other_embed.

    If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
    """

    # patch by Richard So [https://twitter.com/richardjeanso) (thanks!) to update this code for new version of gensim
    # base_embed.init_sims(replace=True)
    # other_embed.init_sims(replace=True)

    # make sure vocabulary and indices are aligned
    in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)
    
    # re-filling the normed vectors
    in_base_embed.wv.fill_norms(force=True)
    in_other_embed.wv.fill_norms(force=True)

    # get the (normalized) embedding matrices
    base_vecs = in_base_embed.wv.get_normed_vectors()
    other_vecs = in_other_embed.wv.get_normed_vectors()

    # just a matrix dot product with numpy
    m = other_vecs.T.dot(base_vecs) 
    # SVD method from numpy
    u, _, v = np.linalg.svd(m)
    # another matrix operation
    ortho = u.dot(v) 
    # Replace original array with modified one, i.e. multiplying the embedding matrix by "ortho"
    other_embed.wv.vectors = (other_embed.wv.vectors).dot(ortho)    
    
    return other_embed

def intersection_align_gensim(m1, m2, words=None):
    """
    Intersect two gensim models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocab_m1 = set(m1.wv.index_to_key)
    vocab_m2 = set(m2.wv.index_to_key)

    # Find the common vocabulary
    common_vocab = vocab_m1 & vocab_m2
    if words: common_vocab &= set(words)

    # If no alignment necessary because vocab is identical...
    if not vocab_m1 - common_vocab and not vocab_m2 - common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.wv.get_vecattr(w, "count") + m2.wv.get_vecattr(w, "count"), reverse=True)
    # print(len(common_vocab))

    # Then for each model...
    for m in [m1, m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.wv.key_to_index[w] for w in common_vocab]
        old_arr = m.wv.vectors
        new_arr = np.array([old_arr[index] for index in indices])
        m.wv.vectors = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        new_key_to_index = {}
        new_index_to_key = []
        for new_index, key in enumerate(common_vocab):
            new_key_to_index[key] = new_index
            new_index_to_key.append(key)
        m.wv.key_to_index = new_key_to_index
        m.wv.index_to_key = new_index_to_key
        
        print(len(m.wv.key_to_index), len(m.wv.vectors))
        
    return (m1,m2)

In [27]:
# Ausrichtung des "After-Corona-Modells" an das "Before-Corona-Modell"

w2v_ac_al = smart_procrustes_align_gensim(w2v_bc, w2v_ac)

1836 1836
1836 1836


In [28]:
# speichern
w2v_ac_al.save(os.path.join('../trained_models', 'w2v_ac_al_querdenker.model'))

## Cosinus-Ähnlichkeit zwischen den Vektoren der beiden Modelle

In [29]:
# laden
w2v_ac_al = Word2Vec.load(os.path.join('../trained_models', 'w2v_ac_al_querdenker.model'))

#### Exploration

In [30]:
# ähnliche Wörter zu 'querdenker' nach Covid-19
w2v_ac_al.wv.most_similar(positive=['querdenker'], topn=25)

[('märz', 0.997990071773529),
 ('querdenken', 0.9979155659675598),
 ('regensburg', 0.9978930950164795),
 ('dezember', 0.9978514909744263),
 ('ring', 0.997846782207489),
 ('behörde', 0.997784435749054),
 ('gericht', 0.997765302658081),
 ('oktober', 0.9977644085884094),
 ('kommend', 0.9977535009384155),
 ('anmeldung', 0.9977402687072754),
 ('zunächst', 0.9977262616157532),
 ('erfurt', 0.9977249503135681),
 ('gestern', 0.9977195858955383),
 ('abhalten', 0.997704029083252),
 ('erwarten', 0.9976914525032043),
 ('laden', 0.9976881742477417),
 ('arbeit', 0.9976617097854614),
 ('gleich', 0.9976603388786316),
 ('nachdem', 0.9976599216461182),
 ('michael', 0.9976365566253662),
 ('gottesdienst', 0.9976306557655334),
 ('vergangen', 0.9976276159286499),
 ('treffpunkt', 0.9976268410682678),
 ('dienstagabend', 0.9976258873939514),
 ('markt', 0.9976199269294739)]

In [31]:
vector_querdenker_bc = w2v_bc.wv['querdenker']  
vector_querdenker_ac_al = w2v_ac_al.wv['querdenker'] 

cosine_querdenker = 1 - spatial.distance.cosine(vector_querdenker_bc, vector_querdenker_ac_al)
cosine_querdenker

0.9702180027961731

In [32]:
# Versuch mit unverfänglichen Wörtern

vector_deutschland_bc = w2v_bc.wv['deutschland']  
vector_deutschland_ac_al = w2v_ac_al.wv['deutschland'] 

cosine_deutschland = 1 - spatial.distance.cosine(vector_deutschland_bc, vector_deutschland_ac_al)
cosine_deutschland

0.9988300204277039

In [33]:
vector_nachricht_bc = w2v_bc.wv['nachricht']  
vector_nachricht_ac_al = w2v_ac_al.wv['nachricht'] 

cosine_nachricht = 1 - spatial.distance.cosine(vector_nachricht_bc, vector_nachricht_ac_al)
cosine_nachricht

0.9990805387496948

In [34]:
vector_schreiben_bc = w2v_bc.wv['schreiben']  
vector_schreiben_ac_al = w2v_ac_al.wv['schreiben'] 

cosine_schreiben = 1 - spatial.distance.cosine(vector_schreiben_bc, vector_schreiben_ac_al)
cosine_schreiben

0.9945586323738098

In [35]:
vector_sagen_bc = w2v_bc.wv['sagen']  
vector_sagen_ac_al = w2v_ac_al.wv['sagen'] 

cosine_sagen = 1 - spatial.distance.cosine(vector_sagen_bc, vector_sagen_ac_al)
cosine_sagen

0.9937768578529358

In [36]:
vector_mann_bc = w2v_bc.wv['mann']  
vector_mann_ac_al = w2v_ac_al.wv['mann'] 

cosine_mann = 1 - spatial.distance.cosine(vector_mann_bc, vector_mann_ac_al)
cosine_mann

0.9949069619178772

In [37]:
vector_frau_bc = w2v_bc.wv['frau']  
vector_frau_ac_al = w2v_ac_al.wv['frau'] 

cosine_frau = 1 - spatial.distance.cosine(vector_frau_bc, vector_frau_ac_al)
cosine_frau

0.9974777698516846

In [38]:
vector_mensch_bc = w2v_bc.wv['mensch']  
vector_mensch_ac_al = w2v_ac_al.wv['mensch'] 

cosine_mensch = 1 - spatial.distance.cosine(vector_mensch_bc, vector_mensch_ac_al)
cosine_mensch

0.9921406507492065

In [39]:
vector_montag_bc = w2v_bc.wv['montag']  
vector_montag_ac_al = w2v_ac_al.wv['montag'] 

cosine_montag = 1 - spatial.distance.cosine(vector_montag_bc, vector_montag_ac_al)
cosine_montag

0.9992777109146118

### durchschnittliche Cosinus-Ähnlichkeit, Standardweichung und 'Normalbereich'

- von allen Wörtern/Vektoren, die in beiden Modellen vorkommen jeweils die Cosinus-Ähnlichkeit berechnen (also zueinander zwischen den Modellen)
- durchschnittliche Ähnlichkeit und Standardabweichung berechnen, um einen 'Normalbereich' zu ermitteln
- liegt der Wert der Cosinus-Ähnlichkeit (desselben Lemmas zwischen den Modellen) unter dem Normalbereich?

In [48]:
# alle Wörter, die in beiden Modellen vorkommmen

vocab_bc = set(w2v_bc.wv.index_to_key)
vocab_ac_al = set(w2v_ac_al.wv.index_to_key)

common_vocab = vocab_bc & vocab_ac_al

In [50]:
len(common_vocab)

1836

In [52]:
# Cosinus-Ähnlichkeit zwischen dem Wort in den beiden Modellen berechnen

cosines = {}

for word in common_vocab:
    vector_bc = w2v_bc.wv[word]  
    vector_ac_al = w2v_ac_al.wv[word] 
    cosine = 1 - spatial.distance.cosine(vector_bc, vector_ac_al)
    cosines[word] = cosine

In [59]:
cosine_df = pd.DataFrame.from_dict(cosines, orient='index', columns=['cosine'])
cosine_df

,cosine
bunt,0.999375
rainer,0.987377
seit,0.977072
offiziell,0.999493
bringen,0.986610
...,...
bewegung,0.997704
ruhig,0.999438
zentrum,0.999403
funktionieren,0.999346


In [62]:
# durchschnittliche Cosinus-Ähnlichkeit der Wörter der beiden Modelle 

statistics.mean(cosine_df['cosine'])

0.9970018906206347

In [63]:
# Standardabweichung

statistics.stdev(cosine_df['cosine'])

0.013675555057604372

In [95]:
# 'Normalbereich' der Cosinus-Ähnlichkeit

border = statistics.mean(cosine_df['cosine']) - statistics.stdev(cosine_df['cosine'])

print('der Normalbereich der Cosinus-Ähnlichkeit liegt über', border)

der Normalbereich der Cosinus-Ähnlichkeit liegt über 0.9840054556563897


In [96]:
# Standardabweichung erscheint hoch
# DF aufsteigend sortieren und die ersten 30 Zeilen ausgeben, um die 'Ausreißer' anzuschauen

cosine_df.sort_values(by=['cosine'], axis=0).iloc[:30]

,cosine
u,0.683810
zweifler_bibelmüde,0.741480
humanist_atheist,0.744527
offene_gesprächsrunde,0.801731
rolandstr,0.852505
etc,0.915169
bürgerwache,0.916220
ab_uhr,0.936913
idee,0.945201
i,0.953564


--> Versuch: Ausreißer (hier mal alle unter 0,9) rauslöschen und erneut den Normalbereich berechnen

In [99]:
cosine_df_new = cosine_df.sort_values(by=['cosine'], axis=0).iloc[6:,:]
cosine_df_new.head()

,cosine
bürgerwache,0.916220
ab_uhr,0.936913
idee,0.945201
i,0.953564
partei,0.956494


In [100]:
statistics.mean(cosine_df_new['cosine']) 

0.9976810107139942

In [101]:
statistics.stdev(cosine_df_new['cosine'])

0.0052027336942286305

In [102]:
# neuer 'Normalbereich' der Cosinus-Ähnlichkeit

border = statistics.mean(cosine_df_new['cosine']) - statistics.stdev(cosine_df_new['cosine'])

print('der Normalbereich der Cosinus-Ähnlichkeit liegt über', border)

der Normalbereich der Cosinus-Ähnlichkeit liegt über 0.9924782770197655


### Ergebnis

In [103]:
cosine_querdenker < border

True

Die Cosinus-Ähnlichkeit zwischen den beiden Vektoren für 'Querdenker' liegt unterhalb des Normalbereichs. <br>
Die Hypothese kann als verifiziert gelten. 